In [1]:
import pandas as pd

In [2]:
names = pd.read_json('yelp-names.json')
addresses = pd.read_json('yelp-addresses.json')
data = names.merge(addresses, left_index=True, right_index=True)

In [3]:
# adding paris to all addresses in order to be able to get gps coordinates
data['address'] = data['address'].astype(str) + ' Paris'

In [4]:
import requests

def Coords():
    coord_list = []
    for index, row in data.iterrows():
        address = row['address']
        api = 'http://api-adresse.data.gouv.fr/search/'
        response = requests.get(api, params={'q': address, 'limit': 1})
        response2 = response.json()
        coord = response2['features'][0]['geometry']['coordinates'] #getting coordinates, wrong order though!
        coord.reverse()
        coord_list.append(coord)

    return coord_list

In [8]:
coord_list = Coords()
data['Coordinates'] = coord_list
data['Coordinates'] = data['Coordinates'].astype(str).apply(lambda x: x.replace('[','').replace(']','')) 
data['Lat'], data['Long'] = data['Coordinates'].str.split(',', 1).str
data['Lat'] = data['Lat'].astype(float)
data['Long'] = data['Long'].astype(float)
data

,name,address,Coordinates,Lat,Long
0,Bubbolitas - Bubble Tea Bar,17 rue Quincampoix Paris,"48.859825, 2.349809",48.859825,2.349809
1,Zen Zoo - Bar à Thé,2 rue Chabanais Paris,"48.866964, 2.336703",48.866964,2.336703
2,Chatime,33 rue de Richelieu Paris,"48.865288, 2.33657",48.865288,2.336570
3,Bubble Tea Cup,164 rue Saint-Martin Paris,"48.862047, 2.351959",48.862047,2.351959
4,Yinki,44 avenue d'Ivry Paris,"48.823457, 2.365802",48.823457,2.365802
...,...,...,...,...,...
98,Panda Crêpe,7 rue Alexandre Coutureau Paris,"48.846276, 2.215182",48.846276,2.215182
99,Bubble Wrap,1663 avenue Roger Salengro Paris,"48.904292, 2.312312",48.904292,2.312312
100,Chatime,9 rue Godot de Mauroy Paris,"48.870809, 2.32679",48.870809,2.326790
101,J’Suis là,160 grande rue Paris,"45.99348, 4.102962",45.993480,4.102962


In [9]:
import plotly.express as px

# Note: only displays "languages" since most dialects don't have GPS coordinates
fig = px.scatter_mapbox(data_frame=data,
                        lat="Lat",
                        lon="Long",
                        hover_name="name",
                        zoom=0,
                        mapbox_style="open-street-map",
                        template='none',
)

# Layout paramaters that are not definable in px.scatter_mapbox
fig.update_layout(
    title_font_size=35,
    legend_font_size=20,
    legend_tracegroupgap=20,
    legend_title_font_size=20
)

fig.show()